# Regression with Crab Age Dataset

<b><i style="font-size:20px">This project made for a <a href="https://www.kaggle.com/competitions/playground-series-s3e16/overview">Kaggle Competition<a/></i></b><br>
Explonations from Kaggle:<br>
Welcome to the 2023 Kaggle Playground Series! Thank you to everyone who participated in and contributed to Season 3 Playground Series so far!

Your Goal: For this Episode of the Series, your task is to use regression to predict the age of crabs given physical attributes. Good luck!

In this project I will try to find best model to predict the age of the crabs. Deep Learning models are a little bit better than classical machine learning models normally expect for a few situations. For example if your dataset has one thousand rows or lower rows. We will explore the dataset and see. Let's start.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv('/kaggle/input/playground-series-s3e16/train.csv')
test=pd.read_csv('/kaggle/input/playground-series-s3e16/test.csv')
submission=pd.read_csv('/kaggle/input/playground-series-s3e16/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
train=pd.get_dummies(train)

In [ ]:
#Correlation heatmap
import seaborn as sns
sns.heatmap(train.corr(numeric_only=True),annot=True)

In [ ]:
abs(train.corr(numeric_only=True)['Age']).sort_values(ascending=False)

In [ ]:
# I select columns with high correlation with target
train=train[["id","Shell Weight","Height","Diameter","Length","Weight","Age"]]

In [ ]:
#Selecting independent and dependent features
x=train.drop(['Age','id'],axis=1)
y=train['Age']

In [ ]:
#This is a function that try to find the best classical machine learning algorithm for the data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns",100)

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge,Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error


def algo_test(x,y):
        L=LinearRegression()
        R=Ridge()
        Lass=Lasso()
        E=ElasticNet()
        ETR=ExtraTreeRegressor()
        GBR=GradientBoostingRegressor()
        kn=KNeighborsRegressor()
        dt=DecisionTreeRegressor()
        xgb=XGBRegressor()
       
        
        algos=[L,R,Lass,E,ETR,GBR,kn,dt,xgb]
        algo_names=['Linear','Ridge','Lasso','ElasticNet','Extra Tree','Gradient Boosting','KNeighborsRegressor','Decision Tree','XGBRegressor']
        
        x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=.2,random_state=42)
        
        r_squared= []
        rmse= []
        mae= []
        
    
        result=pd.DataFrame(columns=['R_Squared','RMSE','MAE'],index=algo_names)
        
        
        for algo in algos:
            p=algo.fit(x_train,y_train).predict(x_test)
            r_squared.append(r2_score(y_test,p))
            rmse.append(mean_squared_error(y_test,p)**.5)
            mae.append(mean_absolute_error(y_test,p))
        
            

        
        result.R_Squared=r_squared
        result.RMSE=rmse
        result.MAE=mae
        
        rtable=result.sort_values('R_Squared',ascending=False)
        return rtable

In [ ]:
algo_test(x,y)

R2_Scores are not satisfactory. But if you've read the [kaggle competition](https://www.kaggle.com/competitions/playground-series-s3e16/overview) you will find that the metric is mean absolute error. That's why I will try to find the best model taking advntage of mean absolute error.

In [ ]:
train["Age"].mean()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [ ]:
model=Sequential()
model.add(Dense(120, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(80, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(30, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(4, activation="relu"))
model.add(Dense(1))
model.compile(loss="mse", optimizer="adam")

In [ ]:
history=model.fit(x,y,validation_split=0.2,epochs=100,batch_size=32,verbose=1)

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=.2,random_state=42)
prediction=model.predict(x_test)

print(r2_score(y_test,prediction))
print(mean_squared_error(y_test,prediction)**.5)
print(mean_absolute_error(y_test,prediction))

As I said above deep learning isn't better than classical machine learning. That may have different reasons. But I think that the main reason is my datset's structure.

In [ ]:
test=test[["id","Shell Weight","Height","Diameter","Length","Weight"]]

In [ ]:
test.head()

In [ ]:
train.head()

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
GBR=GradientBoostingRegressor()
model=GBR.fit(x,y)
p_data=test.drop(["id","Age"],axis=1)
predictions=model.predict(p_data)
test['Age']=predictions

In [ ]:
test.head()

In [ ]:
model.predict(np.array([8.348928, 0.3750, 1.1750, 1.0500, 28.973189]).reshape(1, -1))

In [ ]:
submission.head()

In [ ]:
submission['Age']=test['Age']
submission.to_csv('submission.csv',index=False)
submission.head()

In [ ]:
submission.tail()

# Trying Without Outliers

The reason can be that there are outliers in the dataset. I will try to remove them and see if it improves the model.

In [ ]:
sns.boxplot(train["Shell Weight"]);

In [ ]:
train.head()

In [ ]:
sns.boxplot(train["Height"]);

In [ ]:
outliers=train.quantile(q=.97, numeric_only=True)

In [ ]:
train=train[(train['Shell Weight']<outliers['Shell Weight'])]
train=train[(train['Height']<outliers['Height'])]
train=train[(train['Diameter']<outliers['Diameter'])]
train=train[(train['Length']<outliers['Length'])]
train=train[(train['Weight']<outliers['Weight'])]

In [ ]:
train.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
x=train.drop(['Age','id'],axis=1)
y=train['Age']
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=.2,random_state=42)

GBR=GradientBoostingRegressor()
model=GBR.fit(x,y)
p_data=test.drop(["id","Age"],axis=1)
predictions=model.predict(p_data)
test['Age']=predictions
submission['Age']=test['Age']
submission.to_csv('submission.csv',index=False)

In [ ]:
model.predict(np.array([8.348928, 0.3750, 1.1750, 1.0500, 28.973189]).reshape(1, -1))

In [ ]:
algo_test(x,y)

In [ ]:
import pickle
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [ ]:
with open('model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)
print(loaded_model)

To conclusion I think that outliers didn't influnce the model. Maybe if I have manipulated the dataset I will get better results. You could try that or another thinks to improve the model. Also I can say that metrics aren't sufficated for understand the model’s performance. That's why I think that test my model with real life is a good choice to understand my model's performance exploratively. I will build a Streamlit app to do that. You can see that in [my huggingface profile](https://huggingface.co/emirkaanozdemr).